In [1]:
import config
import geopandas as gpd
import json
import networkx as nx
import numpy as np
import os
import osmnx as ox
import pandas as pd
import time

ox.config(use_cache=True,
          log_file=True,
          log_console=True,
          log_filename='calculate-cities',
          cache_folder=config.cities_cache_folder)

print(ox.__version__)
print(nx.__version__)

0.8.1
2.1


In [2]:
graphml_folder = config.cities_graphml_folder
places_folder = 'input_data/places' #tiger place shapefiles
stats_folder = config.cities_stats_folder

In [3]:
places = []
for state_folder in os.listdir(graphml_folder):
    for city_file in os.listdir('{}/{}'.format(graphml_folder, state_folder)):

        data = {}
        data['state_folder'] = state_folder
        data['state_fips'] = state_folder.split('_')[0]
        data['state'] = state_folder.split('_')[1]
        data['city_file'] = city_file
        data['geoid'] = city_file.split('_')[0]
        data['city'] = city_file.strip('_{}'.format(data['geoid'])).replace('.graphml', '').replace('_', ' ')
        places.append(data)

df = pd.DataFrame(places)

## Get land area data from shapefiles and merge into DataFrame

In [4]:
# load each state shapefile and get the geoid and aland for each city row
gdf = gpd.GeoDataFrame()
for state_fips in df['state_fips'].unique():
    path = '{}/tl_2017_{}_place'.format(places_folder, state_fips)
    gdf = gdf.append(gpd.read_file(path)[['GEOID', 'ALAND']])

# merge aland values into dataframe, on geoid
gdf = gdf.rename(columns=str.lower)
df = pd.merge(df, gdf, how='left', on='geoid')

## Load graph and calculate stats for each city

In [5]:
def load_graph_get_stats(row):
    
    try:
        start_time = time.time()
        folder = '{}/{}'.format(graphml_folder, row['state_folder'])
        G = ox.load_graphml(filename=row['city_file'], folder=folder)
        
        stats = ox.basic_stats(G, area=row['aland'])
        
        # unpack k-counts and k-proportion dicts into individiual keys:values
        for k, count in stats['streets_per_node_counts'].items():
            stats['int_{}_streets_count'.format(k)] = count
        for k, proportion in stats['streets_per_node_proportion'].items():
            stats['int_{}_streets_prop'.format(k)] = proportion
            
        # calculate/drop the extended stats that have values per node
        extended_stats = ox.extended_stats(G)
        se = pd.Series(extended_stats)
        se = se.drop(['avg_neighbor_degree', 'avg_weighted_neighbor_degree', 'clustering_coefficient',
                      'clustering_coefficient_weighted', 'degree_centrality', 'pagerank'])
        extended_stats_clean = se.to_dict()
        
        for key in extended_stats_clean:
            stats[key] = extended_stats_clean[key]
        
        stats['area_km'] = row['aland'] / 1e6        
        stats['city'] = row['city']
        stats['state'] = row['state']
        stats['geoid'] = row['geoid']
        stats['area'] = row['aland']
        stats['time'] = time.time()-start_time
        
        return pd.Series(stats)

    except Exception as e:
        print('{}, {} failed: {}'.format(row['city'], row['state'], e))
        return pd.Series()

In [6]:
#sample = list(range(0, len(df), int(len(df)/100)))
#stats = df.iloc[sample].apply(load_graph_get_stats, axis=1)
stats_temp = df.apply(load_graph_get_stats, axis=1)
stats_temp.shape

g:\geoff\dropbox\documents\school\phd\projects\code\osmnx-repos\osmnx\osmnx\stats.py:201: RuntimeWarning: divide by zero encountered in double_scalars
  circuity_avg = edge_length_total / gc_distances.sum()


Lake Aluma, OK failed: float division by zero
Ophir, UT failed: float division by zero


(19642, 67)

In [7]:
stats_temp['time'].sum()

6466.0165383815765

## Clean up the dataframe

In [8]:
stats = stats_temp.copy()

In [9]:
# stuff to drop
cols_to_drop = ['area', 'time', 'streets_per_node_counts', 'streets_per_node_proportion', 
                'pagerank_max_node', 'pagerank_min_node', 'clean_intersection_count',
                'clean_intersection_density_km']

In [10]:
cols_to_rename = {}
for col in stats.columns:
    if 'int_' in col:
        n = col.split('_')[1]
        if n not in ['1', '3', '4']:
            cols_to_drop.append(col)
        else:
            suffix = 'count' if 'count' in col else 'proportion'
            cols_to_rename[col] = 'intersect_{}way_{}'.format(n, suffix)
            
stats = stats.drop(cols_to_drop, axis=1)

In [11]:
# rename these to friendlier names
cols_to_rename['clustering_coefficient_avg'] = 'cluster_coeff_avg'
cols_to_rename['clustering_coefficient_weighted_avg'] = 'cluster_coeff_weighted_avg'
cols_to_rename['intersection_density_km'] = 'intersect_density_km'
cols_to_rename['intersect_1way_count'] = 'dead_end_count'
cols_to_rename['intersect_1way_proportion'] = 'dead_end_proportion'
cols_to_rename['m'] = 'edge_count'
cols_to_rename['n'] = 'node_count'
stats = stats.rename(columns=cols_to_rename)
stats = stats.rename(columns=cols_to_rename)

In [12]:
# drop anything lacking a GEOID
stats = stats.dropna(subset=['geoid'])

In [13]:
# make these integers
cols_int = ['intersection_count', 'edge_length_total', 'edge_count', 'node_count', 'street_segments_count']
stats[cols_int] = stats[cols_int].astype(np.int64)

In [14]:
# make city, state, geoid at left of df
cols = stats.columns.tolist()
cols.insert(0, cols.pop(cols.index('city')))
cols.insert(1, cols.pop(cols.index('state')))
cols.insert(2, cols.pop(cols.index('geoid')))
stats = stats.reindex(columns=cols)

## View the results

In [15]:
stats.shape

(19640, 33)

In [16]:
stats.columns

Index(['city', 'state', 'geoid', 'area_km', 'avg_neighbor_degree_avg',
       'avg_weighted_neighbor_degree_avg', 'circuity_avg', 'cluster_coeff_avg',
       'cluster_coeff_weighted_avg', 'degree_centrality_avg',
       'edge_density_km', 'edge_length_avg', 'edge_length_total',
       'dead_end_count', 'dead_end_proportion', 'intersect_3way_count',
       'intersect_3way_proportion', 'intersect_4way_count',
       'intersect_4way_proportion', 'intersection_count',
       'intersect_density_km', 'k_avg', 'edge_count', 'node_count',
       'node_density_km', 'pagerank_max', 'pagerank_min',
       'self_loop_proportion', 'street_density_km', 'street_length_avg',
       'street_length_total', 'street_segments_count', 'streets_per_node_avg'],
      dtype='object')

In [17]:
stats.head()

,city,state,geoid,area_km,avg_neighbor_degree_avg,avg_weighted_neighbor_degree_avg,circuity_avg,cluster_coeff_avg,cluster_coeff_weighted_avg,degree_centrality_avg,...,node_count,node_density_km,pagerank_max,pagerank_min,self_loop_proportion,street_density_km,street_length_avg,street_length_total,street_segments_count,streets_per_node_avg
0,Abbeville,AL,0100124,40.255362,2.762393,0.035414,1.068647,0.044160,0.002017,0.014098,...,351,8.719335,0.008352,0.000432,0.013857,2257.800737,200.637055,90888.586,453,2.598291
1,Adamsville,AL,0100460,65.211854,2.720997,0.028521,1.099474,0.042593,0.002074,0.007590,...,612,9.384797,0.006342,0.000255,0.001409,2345.979904,206.180187,152985.699,742,2.694444
2,Addison,AL,0100484,9.753292,2.760518,0.021447,1.056957,0.064725,0.005696,0.046450,...,103,10.560537,0.023818,0.001468,0.008197,2881.086714,228.455935,28100.080,123,2.543689
3,Akron,AL,0100676,1.776163,3.248485,0.036485,1.075132,0.093939,0.021408,0.101010,...,55,30.965626,0.039553,0.002770,0.000000,5186.226715,122.821120,9211.584,75,2.836364
4,Alabaster,AL,0100820,65.217462,2.717090,0.028984,1.091252,0.029031,0.001685,0.002445,...,1813,27.799303,0.001746,0.000084,0.004482,5247.764962,164.068031,342245.912,2086,2.357970


## Save to disk

In [18]:
if not os.path.exists(stats_folder):
    os.makedirs(stats_folder)
output_path = '{}/cities-stats.csv'.format(stats_folder)
stats.to_csv(output_path, encoding='utf-8', index=False)